Notebook for playing around with cuda, first using libs like pytorch and keops, and then using real cuda kernel code with pyCUDA.

# Nearest neighbor with cuda

Idea : test the K-nn algorithm optimization tricks in CUDA introduced in the [7th MVA class of Jean Feydy](https://www.jeanfeydy.com/Teaching/index.html)

x_i, y_i are arrays ;
We want the nearest neighbor in y_i for each term of x_i.

## 1/ Pytorch implementation (cpu)

In [43]:
import torch
import numpy as np

n = 5000
m = 2500
d = 100

#take yi = xi for nn inside the same group
xi = torch.rand((n,d))
yi = torch.rand((m,d))


def nn_cpu(xi, yi):
  deltas = xi.view((1,n,d)) - yi.view((m,1,d))
  distances = torch.sum(deltas**2, dim=2)
  return distances.argmin(dim=0)


In [79]:
torch.cuda.empty_cache()

Strange behavior from timeit command, we will use the torch timer anyways, which is adapted to cuda code.

In [48]:
import torch.utils.benchmark as benchmark

t0 = benchmark.Timer(
    stmt='nn_cpu(xi, yi)',
    setup="""
    def nn_cpu(xi, yi):
      deltas = xi.view((1,n,d)) - yi.view((m,1,d))
      distances = torch.sum(deltas**2, dim=2)
      return distances.argmin(dim=0)""",
    globals={'xi': xi,"yi":yi,"n":n,"m":m,"d":d})

print(t0.timeit(1))
#too slow !

nn_cpu(xi, yi)
setup:
  def nn_cpu(xi, yi):
    deltas = xi.view((1,n,d)) - yi.view((m,1,d))
    distances = torch.sum(deltas**2, dim=2)
    return distances.argmin(dim=0)

  8.37 s
  1 measurement, 1 runs , 1 thread


## 2/Pytorch implementation (gpu)

In [76]:
import torch
import numpy as np

device = "cuda"

#take yi = xi for nn inside the same group
xi = torch.rand((n,d)).to(device)
yi = torch.rand((m,d)).to(device)

def nn_gpu(xi,yi):
  deltas = xi.view((1,n,d)) - yi.view((m,1,d))
  distances = torch.sum(deltas**2, dim=2)
  return distances.argmin(dim=0)



In [89]:
t1 = benchmark.Timer(
    stmt='nn_gpu(xi, yi)',
    setup="""
    xi = torch.rand((n,d)).cuda()
    yi = torch.rand((m,d)).cuda()

    def nn_gpu(xi,yi):
      deltas = xi.view((1,n,d)) - yi.view((m,1,d))
      distances = torch.sum(deltas**2, dim=2)
      return distances.argmin(dim=0)
""",
    globals={"n":n,"m":m,"d":d})

print(t1.timeit(10))


nn_gpu(xi, yi)
setup:
  xi = torch.rand((n,d)).cuda()
  yi = torch.rand((m,d)).cuda()

  def nn_gpu(xi,yi):
    deltas = xi.view((1,n,d)) - yi.view((m,1,d))
    distances = torch.sum(deltas**2, dim=2)
    return distances.argmin(dim=0)

  107.02 ms
  1 measurement, 10 runs , 1 thread


This is better

## Pytorch implementation optimized

Using the fact that $\|x_k - y_j \|^2 = x_k^2 +y_j^2 - 2 x_k y_j$

Thus, $\text{argmin}_j \|x_k - y_j \|^2 = \text{argmin}_j -2x_k y_j +y_j^2$

We can pre-compute the y_j to save memory inside the GPU, in order to speed up computation and avoid unnecessary memory transfers between the GPU's global memory and shared memory


In [18]:
import torch
import numpy as np

device = "cuda"

#take yi = xi for nn inside the same group
xi = torch.rand((n,d)).to(device)
yi = torch.rand((m,d)).to(device)

def nn_gpu_opti(xi,yi):
  y_sq = torch.sum(yi**2, dim=1)
  dot = -2 * xi@yi.T
  dist_minus_x_sq = y_sq.view((1,m)) + dot
  nn = dist_minus_x_sq.argmin(dim=0)
  return nn

In [91]:
t3 = benchmark.Timer(
    stmt='nn_gpu_opti(xi, yi)',
    setup="""
    xi = torch.rand((n,d)).cuda()
    yi = torch.rand((m,d)).cuda()

    def nn_gpu_opti(xi,yi):
      y_sq = torch.sum(yi**2, dim=1)
      dot = -2 * xi@yi.T
      dist_minus_x_sq = y_sq.view((1,m)) + dot
      nn = dist_minus_x_sq.argmin(dim=0)
      return nn
""",
    globals={"n":n,"m":m,"d":d})

print(t3.timeit(100))


nn_gpu_opti(xi, yi)
setup:
  xi = torch.rand((n,d)).cuda()
  yi = torch.rand((m,d)).cuda()

  def nn_gpu_opti(xi,yi):
    y_sq = torch.sum(yi**2, dim=1)
    dot = -2 * xi@yi.T
    dist_minus_x_sq = y_sq.view((1,m)) + dot
    nn = dist_minus_x_sq.argmin(dim=0)
    return nn

  1.92 ms
  1 measurement, 100 runs , 1 thread


This is A LOT faster than the previous one, we are nearly at the hardware limit because for d >= 100 with this algorithm, the memory transfer operations (inside the GPU from the global memory to the threads) are no longer the bottleneck (since one memory transfer ~100 GPU operations).
Now let us try without even storing the distances, using the KeOos library.


## Using KeOps

In [45]:
!pip install pykeops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 12.2 MB/s eta 0:00:00
  Created wheel for pykeops: filename=pykeops-2.2.3-py3-none-any.whl size=118636 sha256=7c0711afdd9a68c092c42ad3f4ff687b43a7bc5da73609693b1cc518343200a4
  Stored in directory: /root/.cache/pip/wheels/37/8b/c7/25e5194a7138fd564c3ef3e275ae0155c207cd85d7ab347817
  Created wheel for keopscore: filename=keopscore-2.2.3-py3-none-any.whl size=172483 sha256=9c5a14c199dde5b7c3cce4a3d2eb7eee1f7549de072a954da7ca8c9b6f704f32
  Stored in directory: /root/.cache/pip/wheels/3d/d8/ee/29900acfbbd7ee0f3b05981d3d172baad4c3b5d40cbf4c5d74
Successfully built pykeops keopscore


In [46]:
from pykeops.torch import LazyTensor
xi_lazy = LazyTensor(xi.view((1,n,d)))
yi_lazy = LazyTensor(yi.view((m,1,d)))

D_ij = ((xi_lazy - yi_lazy) ** 2).sum(-1)
ind_nn = D_ij.argKmin(1, dim=1)


[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK
[KeOps] Generating code for ArgKMin_Reduction reduction (with parameters 0) of formula Sum((a-b)**2) with a=Var(0,100,1), b=Var(1,100,0) ... OK


In [49]:
t4 = benchmark.Timer(
    stmt='nn_gpu(xi_lazy, yi_lazy)',
    setup="""
    def nn_gpu(xi_lazy,yi_lazy):
      D_ij = ((xi_lazy - yi_lazy) ** 2).sum(-1)
      ind_nn = D_ij.argKmin(1, dim=1)
      return ind_nn
""",
    globals={'xi_lazy': xi_lazy,"yi_lazy":yi_lazy,"n":n,"m":m,"d":d})

print(t4.timeit(1))


nn_gpu(xi_lazy, yi_lazy)
setup:
  def nn_gpu(xi_lazy,yi_lazy):
    D_ij = ((xi_lazy - yi_lazy) ** 2).sum(-1)
    ind_nn = D_ij.argKmin(1, dim=1)
    return ind_nn

  12.95 ms
  1 measurement, 1 runs , 1 thread


The fastest algorithm is the handmade, optimized one but the KeOPS lib is still impressive : 10x faster than "naive" pytorch

# More fun with cuda

In [2]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.4/92.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=f9fd31f837d5d22e5605533f44a6e3226d61af9371a91fd7cf1841fef439e3d7
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [13]:
## pyCUDA demo from the documentation : elementwise multiplication

import pycuda.autoinit
import pycuda.driver as drv
import numpy as np

from pycuda.compiler import SourceModule
mod = SourceModule("""
__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
""")

multiply_them = mod.get_function("multiply_them")

a = np.random.randn(400).astype(np.float32)
b = np.random.randn(400).astype(np.float32)

dest = np.zeros_like(a)
multiply_them(
        drv.Out(dest), drv.In(a), drv.In(b),
        block=(400,1,1), grid=(1,1))

print(all(dest == a*b))


True


Test : naive matrix multiplication

In [41]:

# A,B square matrices of dim d
# each core i gets an element of the matrix AB : (AB)_{i,j} = sum(A_{i,k}B_{k,j})
# with i = d * j + k

mod2 = SourceModule("""
__global__ void matmul_naive(float *dest, float *A, float *B,int d)
{
  const int i = threadIdx.x;
  const int j = threadIdx.y;

  float sum = 0;
  for (int l = 0; l < d; l++){
    sum += A[i * d + l] * B[l * d + j];
  }

  dest[i*d + j] = sum;
}
""")

matmul_naive = mod2.get_function("matmul_naive")
d = 10
A = np.random.rand(d,d)
A = A.astype(np.float32)
B = np.random.rand(d,d)
B = B.astype(np.float32)

dest = np.zeros((d,d)).astype(np.float32)

A = np.ascontiguousarray(A)
B = np.ascontiguousarray(B)
dest = np.ascontiguousarray(dest)

matmul_naive(
        drv.Out(dest), drv.In(A), drv.In(B), np.int32(d),
        block=(d,d,1), grid=(1,1,1))


d = dest - A@B
print(np.max(d))


2.3841858e-07


THe algorithm works up to some precisions issues, probably due to float conversion.